In [ ]:
from PVAssess.input_generation import Generate
from PVAssess.output_extraction import Extract

In [ ]:
cluster_name = 'zayahcortright'
cluster_command = ''
material = ''

Generate(cluster_name, cluster_command, material)

##### scp to cluster with bash scripts, run scripts inside of the main folder (the one containing the O, H, OH, and OOH folders)

In [ ]:
Extract("SrCoO3_A.xyz")